## Фина настройка на модел

My PUZL torch version: torch 2.1.0+cu118
Install using: ```!pip install "unsloth[cu118-ampere] @ git+https://github.com/unslothai/unsloth.git" ```
or ```!pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"``` for VAST.AI

__WARNINIG!!!__: use ```sudo apt-get update``` and ```sudo apt-get install python3-dev``` in terminal before running


In [ ]:
%pip install "unsloth[cu118-ampere] @ git+https://github.com/unslothai/unsloth.git"
%pip install tensorboardX

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1800
dtype = None # None for auto detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "hf_***",
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
import json
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

#Prompt formats###########################################################################
# lama_text_row = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

# {Question}<|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>

# {Answer}<|eot_id|>{EOS_TOKEN}"""

# mistral_text_row = f"""<s>[INST]{Question}[/INST]{Answer}</s> {EOS_TOKEN}"""
##########################################################################################
from datasets import load_dataset
dataset = load_dataset("torVik/bglawinstructAlphaMINI", split="train")
def create_text_row(Question, Answer):
    text_row = f"""<s>[INST]{Question}[/INST]{Answer}</s> {EOS_TOKEN}"""
    return text_row

def process_jsonl_file(output_file_path):
    with open(output_file_path, "w", encoding="utf-8") as output_jsonl_file:
        for item in dataset:
            json_object = {
                "text": create_text_row(item["Question"], item["Answer"]),
            }
            output_jsonl_file.write(json.dumps(json_object, ensure_ascii=False) + "\n")

process_jsonl_file("./training_dataset.jsonl")

train_dataset = load_dataset('json', data_files='./training_dataset.jsonl' , split='train')
new_model = "mistral-7b-Instruct-bglawinsv2UNS"

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import tensorboardX

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_batch_size = 6,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 6,
        gradient_accumulation_steps = 2,
        num_train_epochs=3,
        warmup_steps = 0.05,
        learning_rate = 0.0002,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        save_steps=1000,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "mistral-7b-Instruct-bglawinsv2UNS",
        report_to="tensorboard",
        push_to_hub = True
    ),
)

In [ ]:
trainer.train()
trainer.model.save_pretrained(new_model)